# Day 4: Pandas - Data Mastery 

Welcome to **Pandas**, the most powerful library for data manipulation in Python! If NumPy is the foundation, Pandas is the entire building where data science actually happens.

### Why Pandas?
- It provides **DataFrames** - spreadsheet-like structures for easy data handling.
- It can handle **missing data** gracefully.
- It offers powerful **grouping, merging, and aggregation** capabilities.
- It's the go-to tool for **Exploratory Data Analysis (EDA)**.

### Topics Covered:
1. **Series & DataFrames**
2. **Loading & Inspecting Data**
3. **Handling Missing Values**
4. **Filtering & Selection**
5. **GroupBy & Aggregations**
6. **Merging DataFrames**
7. **The Apply Function**
8. **Mini Project: Sales Analysis**

In [ ]:
# First, let's import the essential libraries
import pandas as pd
import numpy as np

# Set display options for better readability
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 100)

## 1. Series & DataFrames

Pandas has two main data structures:
- **Series**: A 1D labeled array (like a column in Excel)
- **DataFrame**: A 2D labeled table (like a spreadsheet)

In [ ]:
# Creating a Series
sales = pd.Series([250, 320, 180, 410], 
                  index=['Jan', 'Feb', 'Mar', 'Apr'],
                  name='Monthly Sales')
print("=== Series ===")
print(sales)
print(f"\nType: {type(sales)}")
print(f"Sum: ${sales.sum()}")
print(f"Mean: ${sales.mean():.2f}")

In [ ]:
# Creating a DataFrame from a dictionary
data = {
    'Product': ['Laptop', 'Phone', 'Tablet', 'Monitor', 'Keyboard'],
    'Category': ['Electronics', 'Electronics', 'Electronics', 'Electronics', 'Accessories'],
    'Price': [1200, 800, 450, 350, 75],
    'Quantity': [50, 150, 80, 60, 200],
    'Rating': [4.5, 4.8, 4.2, 4.0, 4.6]
}

df = pd.DataFrame(data)
print("=== DataFrame ===")
print(df)

## 2. Loading & Inspecting Data

In real-world scenarios, you'll load data from files. Let's create a sample dataset and learn inspection techniques.

In [ ]:
# Let's create a more realistic sales dataset
np.random.seed(42)  # For reproducibility

n_records = 100
regions = ['North', 'South', 'East', 'West']
products = ['Laptop', 'Phone', 'Tablet', 'Headphones', 'Charger']
salespeople = ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve']

sales_data = pd.DataFrame({
    'Date': pd.date_range(start='2024-01-01', periods=n_records, freq='D'),
    'Region': np.random.choice(regions, n_records),
    'Product': np.random.choice(products, n_records),
    'Salesperson': np.random.choice(salespeople, n_records),
    'Units_Sold': np.random.randint(1, 50, n_records),
    'Unit_Price': np.random.choice([50, 100, 250, 500, 1000], n_records),
    'Customer_Rating': np.random.choice([3.0, 3.5, 4.0, 4.5, 5.0, np.nan], n_records)  # Include some NaN
})

# Calculate Revenue
sales_data['Revenue'] = sales_data['Units_Sold'] * sales_data['Unit_Price']

print("Dataset created! Let's inspect it...")

In [ ]:
# Essential inspection methods
print("=== First 5 Rows (head) ===")
print(sales_data.head())

print("\n=== Last 3 Rows (tail) ===")
print(sales_data.tail(3))

In [ ]:
# DataFrame structure and info
print("=== Shape ===")
print(f"Rows: {sales_data.shape[0]}, Columns: {sales_data.shape[1]}")

print("\n=== Column Names ===")
print(sales_data.columns.tolist())

print("\n=== Data Types ===")
print(sales_data.dtypes)

In [ ]:
# Comprehensive info and statistics
print("=== Info ===")
print(sales_data.info())

print("\n=== Statistical Summary ===")
print(sales_data.describe())

## 3. Handling Missing Values

Real-world data is messy. Missing values (`NaN`) are common. Pandas provides tools to detect and handle them.

In [ ]:
# Detecting missing values
print("=== Missing Values Count ===")
print(sales_data.isnull().sum())

print(f"\nTotal missing: {sales_data.isnull().sum().sum()}")
print(f"Percentage missing in Customer_Rating: {sales_data['Customer_Rating'].isnull().mean() * 100:.1f}%")

In [ ]:
# Method 1: Drop rows with missing values
df_dropped = sales_data.dropna()
print(f"Original rows: {len(sales_data)}")
print(f"After dropna: {len(df_dropped)}")

In [ ]:
# Method 2: Fill missing values
# Option A: Fill with a constant
df_filled_constant = sales_data.copy()
df_filled_constant['Customer_Rating'] = df_filled_constant['Customer_Rating'].fillna(0)

# Option B: Fill with mean (most common approach)
df_filled_mean = sales_data.copy()
mean_rating = df_filled_mean['Customer_Rating'].mean()
df_filled_mean['Customer_Rating'] = df_filled_mean['Customer_Rating'].fillna(mean_rating)

print(f"Mean rating used for filling: {mean_rating:.2f}")
print(f"Missing values after fillna: {df_filled_mean['Customer_Rating'].isnull().sum()}")

## 4. Filtering & Selection

Selecting specific rows and columns is fundamental to data analysis.

In [ ]:
# Selecting columns
print("=== Single Column (Series) ===")
print(sales_data['Product'].head())

print("\n=== Multiple Columns (DataFrame) ===")
print(sales_data[['Product', 'Revenue', 'Region']].head())

In [ ]:
# Filtering rows with conditions
print("=== High Revenue Sales (> $5000) ===")
high_revenue = sales_data[sales_data['Revenue'] > 5000]
print(high_revenue.head())
print(f"\nCount: {len(high_revenue)} transactions")

In [ ]:
# Multiple conditions (AND: &, OR: |)
print("=== North Region + High Rating (>= 4.5) ===")
filtered = sales_data[(sales_data['Region'] == 'North') & (sales_data['Customer_Rating'] >= 4.5)]
print(filtered[['Region', 'Product', 'Revenue', 'Customer_Rating']].head())
print(f"\nCount: {len(filtered)} transactions")

In [ ]:
# Using .loc (label-based) and .iloc (index-based)
print("=== Using .loc (by label) ===")
print(sales_data.loc[0:4, ['Product', 'Region', 'Revenue']])

print("\n=== Using .iloc (by position) ===")
print(sales_data.iloc[0:5, [2, 1, 7]])  # First 5 rows, columns at positions 2, 1, 7

## 5. GroupBy & Aggregations

One of the most powerful features! Group data and calculate aggregate statistics.

In [ ]:
# Simple groupby with single aggregation
print("=== Total Revenue by Region ===")
revenue_by_region = sales_data.groupby('Region')['Revenue'].sum().sort_values(ascending=False)
print(revenue_by_region)

In [ ]:
# Multiple aggregations with .agg()
print("=== Revenue Stats by Region ===")
region_stats = sales_data.groupby('Region')['Revenue'].agg(['sum', 'mean', 'count', 'max'])
region_stats.columns = ['Total', 'Average', 'Count', 'Max']
print(region_stats)

In [ ]:
# Grouping by multiple columns
print("=== Revenue by Region & Product ===")
region_product = sales_data.groupby(['Region', 'Product'])['Revenue'].sum().unstack(fill_value=0)
print(region_product)

In [ ]:
# Top performing salespeople
print("=== Top Salespeople by Total Revenue ===")
top_salespeople = sales_data.groupby('Salesperson').agg({
    'Revenue': 'sum',
    'Units_Sold': 'sum',
    'Customer_Rating': 'mean'
}).round(2).sort_values('Revenue', ascending=False)
print(top_salespeople)

## 6. Merging DataFrames

Combine data from multiple sources - similar to SQL JOINs.

In [ ]:
# Create additional DataFrames for merging examples
product_info = pd.DataFrame({
    'Product': ['Laptop', 'Phone', 'Tablet', 'Headphones', 'Charger'],
    'Category': ['Electronics', 'Electronics', 'Electronics', 'Audio', 'Accessories'],
    'Cost': [800, 500, 300, 30, 10]
})

region_targets = pd.DataFrame({
    'Region': ['North', 'South', 'East', 'West'],
    'Target': [50000, 45000, 55000, 40000],
    'Manager': ['John', 'Sarah', 'Mike', 'Lisa']
})

print("=== Product Info ===")
print(product_info)
print("\n=== Region Targets ===")
print(region_targets)

In [ ]:
# Merge sales data with product info
print("=== Merged: Sales + Product Cost ===")
merged = pd.merge(sales_data, product_info[['Product', 'Cost']], on='Product', how='left')
merged['Profit'] = merged['Revenue'] - (merged['Cost'] * merged['Units_Sold'])
print(merged[['Product', 'Revenue', 'Cost', 'Units_Sold', 'Profit']].head(10))

In [ ]:
# Merge types explained
print("=== Merge Types ===")
print("• INNER: Only matching rows (default)")
print("• LEFT:  All rows from left table + matching from right")
print("• RIGHT: All rows from right table + matching from left")
print("• OUTER: All rows from both tables")

# Example: Add region manager to sales data
sales_with_manager = pd.merge(sales_data, region_targets, on='Region', how='left')
print("\n=== Sales with Manager Info ===")
print(sales_with_manager[['Date', 'Region', 'Manager', 'Revenue', 'Target']].head())

## 7. The Apply Function

Apply custom functions to your data - extremely powerful for complex transformations.

In [ ]:
# Apply to a single column
def categorize_revenue(revenue):
    if revenue < 1000:
        return 'Low'
    elif revenue < 5000:
        return 'Medium'
    else:
        return 'High'

sales_data['Revenue_Category'] = sales_data['Revenue'].apply(categorize_revenue)
print("=== Revenue Categories ===")
print(sales_data['Revenue_Category'].value_counts())

In [ ]:
# Apply with lambda (for simple operations)
sales_data['Revenue_K'] = sales_data['Revenue'].apply(lambda x: f"${x/1000:.1f}K")
print("=== Revenue in Thousands ===")
print(sales_data[['Revenue', 'Revenue_K', 'Revenue_Category']].head())

In [ ]:
# Apply to entire row (axis=1)
def create_summary(row):
    return f"{row['Salesperson']} sold {row['Units_Sold']} {row['Product']}(s) in {row['Region']}"

sales_data['Summary'] = sales_data.apply(create_summary, axis=1)
print("=== Transaction Summaries ===")
print(sales_data['Summary'].head())

---

## 🎯 Mini Project: Complete Sales Analysis

**Goal:** Analyze the sales data to find top-performing regions, products, and salespeople.

Let's put everything together!

In [ ]:
# Re-create clean dataset for the project
np.random.seed(42)
n_records = 500

regions = ['North', 'South', 'East', 'West', 'Central']
products = ['Laptop', 'Phone', 'Tablet', 'Headphones', 'Charger', 'Monitor', 'Keyboard']
salespeople = ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve', 'Frank', 'Grace']

sales_df = pd.DataFrame({
    'Date': pd.date_range(start='2024-01-01', periods=n_records, freq='D'),
    'Region': np.random.choice(regions, n_records),
    'Product': np.random.choice(products, n_records),
    'Salesperson': np.random.choice(salespeople, n_records),
    'Units_Sold': np.random.randint(1, 100, n_records),
    'Unit_Price': np.random.choice([25, 50, 100, 250, 500, 750, 1000], n_records),
    'Customer_Rating': np.random.choice([3.0, 3.5, 4.0, 4.5, 5.0, np.nan], n_records, 
                                        p=[0.1, 0.15, 0.25, 0.25, 0.15, 0.1])
})

# Calculated fields
sales_df['Revenue'] = sales_df['Units_Sold'] * sales_df['Unit_Price']
sales_df['Month'] = sales_df['Date'].dt.month_name()

print("📊 Sales Dataset Ready!")
print(f"   Records: {len(sales_df)}")
print(f"   Date Range: {sales_df['Date'].min().date()} to {sales_df['Date'].max().date()}")
print(f"   Total Revenue: ${sales_df['Revenue'].sum():,.0f}")

In [ ]:
# Task 1: Handle Missing Data
print("=" * 50)
print("📋 TASK 1: Data Quality Check")
print("=" * 50)

print("\n🔍 Missing Values:")
print(sales_df.isnull().sum())

# Fill missing ratings with median
median_rating = sales_df['Customer_Rating'].median()
sales_df['Customer_Rating'] = sales_df['Customer_Rating'].fillna(median_rating)
print(f"\n✅ Filled missing ratings with median: {median_rating}")

In [ ]:
# Task 2: Find Top Performing Regions
print("=" * 50)
print("🏆 TASK 2: Top Performing Regions")
print("=" * 50)

region_performance = sales_df.groupby('Region').agg({
    'Revenue': ['sum', 'mean', 'count'],
    'Units_Sold': 'sum',
    'Customer_Rating': 'mean'
}).round(2)

# Flatten column names
region_performance.columns = ['Total_Revenue', 'Avg_Revenue', 'Transactions', 'Total_Units', 'Avg_Rating']
region_performance = region_performance.sort_values('Total_Revenue', ascending=False)

print("\n📈 Region Performance Summary:")
print(region_performance)

winner = region_performance.index[0]
print(f"\n🥇 TOP REGION: {winner} with ${region_performance.loc[winner, 'Total_Revenue']:,.0f} in revenue!")

In [ ]:
# Task 3: Best Selling Products
print("=" * 50)
print("📦 TASK 3: Best Selling Products")
print("=" * 50)

product_performance = sales_df.groupby('Product').agg({
    'Revenue': 'sum',
    'Units_Sold': 'sum',
    'Customer_Rating': 'mean'
}).round(2).sort_values('Revenue', ascending=False)

product_performance['Revenue_Share_%'] = (product_performance['Revenue'] / product_performance['Revenue'].sum() * 100).round(1)

print("\n📊 Product Performance:")
print(product_performance)

top_product = product_performance.index[0]
print(f"\n🏆 BEST SELLER: {top_product} ({product_performance.loc[top_product, 'Revenue_Share_%']}% of total revenue)")

In [ ]:
# Task 4: Top Salespeople
print("=" * 50)
print("👥 TASK 4: Top Salespeople")
print("=" * 50)

top_salespeople = sales_df.groupby('Salesperson').agg({
    'Revenue': 'sum',
    'Units_Sold': 'sum',
    'Customer_Rating': 'mean',
    'Date': 'count'  # Number of transactions
}).round(2)

top_salespeople.columns = ['Total_Revenue', 'Total_Units', 'Avg_Rating', 'Transactions']
top_salespeople['Avg_Sale_Value'] = (top_salespeople['Total_Revenue'] / top_salespeople['Transactions']).round(2)
top_salespeople = top_salespeople.sort_values('Total_Revenue', ascending=False)

print("\n🌟 Salesperson Leaderboard:")
print(top_salespeople)

mvp = top_salespeople.index[0]
print(f"\n🥇 MVP: {mvp} - ${top_salespeople.loc[mvp, 'Total_Revenue']:,.0f} in total sales!")

In [ ]:
# Task 5: Monthly Trends
print("=" * 50)
print("📅 TASK 5: Monthly Revenue Trends")
print("=" * 50)

# Create a proper month order
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

monthly_revenue = sales_df.groupby('Month')['Revenue'].sum()
# Reorder by calendar month
monthly_revenue = monthly_revenue.reindex([m for m in month_order if m in monthly_revenue.index])

print("\n📈 Revenue by Month:")
for month, revenue in monthly_revenue.items():
    bar = '█' * int(revenue / monthly_revenue.max() * 30)
    print(f"{month:12} | {bar} ${revenue:>10,.0f}")

print(f"\n📊 Best Month: {monthly_revenue.idxmax()} (${monthly_revenue.max():,.0f})")
print(f"📉 Slowest Month: {monthly_revenue.idxmin()} (${monthly_revenue.min():,.0f})")

In [ ]:
# Task 6: Cross-Analysis (Region x Product)
print("=" * 50)
print("🔀 TASK 6: Region-Product Matrix")
print("=" * 50)

pivot_table = sales_df.pivot_table(
    values='Revenue', 
    index='Region', 
    columns='Product', 
    aggfunc='sum',
    fill_value=0
)

print("\n💰 Revenue by Region & Product:")
print(pivot_table.round(0).astype(int))

# Find the best combination
best_combo = pivot_table.stack().idxmax()
best_value = pivot_table.stack().max()
print(f"\n🎯 Best Combination: {best_combo[0]} - {best_combo[1]} = ${best_value:,.0f}")

In [ ]:
# Final Summary Report
print("\n" + "=" * 60)
print("🎉 SALES ANALYSIS EXECUTIVE SUMMARY")
print("=" * 60)

total_revenue = sales_df['Revenue'].sum()
total_units = sales_df['Units_Sold'].sum()
avg_rating = sales_df['Customer_Rating'].mean()

print(f"""
📊 OVERALL METRICS:
   • Total Revenue: ${total_revenue:,.0f}
   • Total Units Sold: {total_units:,}
   • Average Customer Rating: {avg_rating:.2f}/5.0
   • Total Transactions: {len(sales_df)}

🏆 TOP PERFORMERS:
   • Best Region: {region_performance.index[0]}
   • Best Product: {product_performance.index[0]}
   • Top Salesperson: {top_salespeople.index[0]}
   • Best Month: {monthly_revenue.idxmax()}

💡 KEY INSIGHTS:
   • {region_performance.index[0]} region generates {(region_performance.iloc[0]['Total_Revenue']/total_revenue*100):.1f}% of all revenue
   • {product_performance.index[0]} is the revenue leader with {product_performance.iloc[0]['Revenue_Share_%']}% share
   • {top_salespeople.index[0]} leads the team with ${top_salespeople.iloc[0]['Total_Revenue']:,.0f} in sales
""")

print("=" * 60)
print("✅ Day 4 Complete! You've mastered Pandas fundamentals!")
print("=" * 60)

---

## 🎯 Practice Exercises

Try these on your own:

1. **Filter high-value transactions**: Find all sales where Revenue > $5,000 AND Rating >= 4.5
2. **Create a new metric**: Add a `Performance_Score` column that combines Revenue and Rating
3. **Time analysis**: Find which day of the week has the highest average sales
4. **Data export**: Save the analysis results to a CSV file using `df.to_csv('filename.csv')`

---

## 📚 Key Takeaways

| Concept | Key Methods |
|---------|-------------|
| **Loading Data** | `pd.read_csv()`, `pd.read_excel()` |
| **Inspecting** | `.head()`, `.info()`, `.describe()`, `.shape` |
| **Missing Values** | `.isnull()`, `.dropna()`, `.fillna()` |
| **Selection** | `df['col']`, `df[['col1', 'col2']]`, `.loc[]`, `.iloc[]` |
| **Filtering** | `df[df['col'] > value]`, `&`, `|` |
| **Aggregation** | `.groupby()`, `.agg()`, `.sum()`, `.mean()` |
| **Merging** | `pd.merge()`, `pd.concat()` |
| **Transformation** | `.apply()`, lambda functions |

---

**Next Up:** Day 5 - Data Visualization with Matplotlib & Seaborn! 📊